In [1]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# モデルごとにグループ化して集計
grouped_df = combined_df.groupby('Model', sort=False).agg({
    'Average Test Score': ['mean', 'std']
}).reset_index()

# カラム名のリネーム
grouped_df.columns = ['Model', 'Average Test Score Mean', 'Average Test Score Std']

# 小数点2位までに丸める
grouped_df['Average Test Score Mean'] = grouped_df['Average Test Score Mean'].round(2)
grouped_df['Average Test Score Std'] = grouped_df['Average Test Score Std'].round(2)

# モデルごとにデータを分割
rbf_df = grouped_df[grouped_df['Model'].str.startswith('RBF')]
linear_df = grouped_df[grouped_df['Model'].str.startswith('LINEAR')]
old_df = grouped_df[grouped_df['Model'].str.startswith('OLD')]

# 各データフレームのインデックスをリセット
rbf_df.reset_index(drop=True, inplace=True)
linear_df.reset_index(drop=True, inplace=True)
old_df.reset_index(drop=True, inplace=True)

# データを横に並べる
final_df = pd.concat([rbf_df, linear_df, old_df], axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



      Model  Average Test Score Mean  Average Test Score Std        Model  \
0  RBF_LQP1                    84.91                    4.61  LINEAR_LQP1   
1   RBF_SQP                    77.87                    5.15   LINEAR_SQP   
2  RBF_LQP2                    62.79                    2.66  LINEAR_LQP2   

   Average Test Score Mean  Average Test Score Std     Model  \
0                    86.15                    3.26  OLD_LQP1   
1                    78.18                    6.05   OLD_SQP   
2                    63.24                    2.37  OLD_LQP2   

   Average Test Score Mean  Average Test Score Std  
0                    86.87                    1.33  
1                    50.45                    0.78  
2                    57.30                    0.94  


In [2]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# 必要なカラムを選択
selected_columns = ['Model', 'Average TNR', 'Average TPR', 'Average Test Score']
combined_df = combined_df[selected_columns]

# モデルごとにデータを分割してリストに保存
models = combined_df['Model'].unique()
model_dfs = [combined_df[combined_df['Model'] == model].reset_index(drop=True) for model in models]

# 各モデルごとにデータフレームを作成し、リストに追加
final_dfs = []
for df in model_dfs:
    model_name = df['Model'].iloc[0]
    avg_tnr_mean = round(df['Average TNR'].mean(), 2)
    avg_tpr_mean = round(df['Average TPR'].mean(), 2)
    avg_test_score_mean = round(df['Average Test Score'].mean(), 2)
    avg_test_score_std = round(df['Average Test Score'].std(), 2)
    
    summary = pd.DataFrame({
        'Model': [model_name + '_MEAN'],
        'Average TNR': [avg_tnr_mean],
        'Average TPR': [avg_tpr_mean],
        'Average Test Score': [avg_test_score_mean],
        'Average Test Score Std': [avg_test_score_std]
    })
    
    df['Average Test Score Std'] = pd.NA  # 標準偏差の列を追加してNaNで埋める
    summary['Average Test Score Std'] = avg_test_score_std  # 標準偏差の列を更新
    
    df = df.drop(columns=['Model'])
    df.columns = [f'{model_name} {col}' for col in df.columns]
    
    summary.columns = [f'{model_name} {col}' for col in summary.columns]
    
    df = pd.concat([df, summary], ignore_index=True)
    
    final_dfs.append(df)

# すべてのモデルのデータフレームを横に連結
final_df = pd.concat(final_dfs, axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



    RBF_LQP1 Average TNR  RBF_LQP1 Average TPR  RBF_LQP1 Average Test Score  \
0                  66.15                 97.26                        81.70   
1                  70.07                 98.70                        84.39   
2                  91.11                 88.00                        89.56   
3                  66.41                 98.30                        82.35   
4                  68.74                 97.70                        83.22   
5                  54.78                 98.93                        76.85   
6                  85.59                 96.70                        91.15   
7                  67.67                 95.78                        81.72   
8                  81.89                 97.30                        89.59   
9                  84.11                 93.04                        88.57   
10                 73.65                 96.17                        84.91   

   RBF_LQP1 Average Test Score Std RBF_LQP1 Model  